# Analisis de datos sobre la evolución de anime

##### En el archivo data-no-mining hemos conseguido, a partir de una API y un DataFrame de Kaggle, un DataFrame con toda la informacion que necesito.
##### Voy a empezar con la limpieza de ese y otro data frame del 2019.
##### Mas tarde voy a comenzar con los insights comparando un DataFrame con otro.

In [297]:
import pandas as pd
import ast
import numpy as np

df_2024 = pd.read_csv('anime_data.csv')
df_2019 = pd.read_csv('Anime.csv')

### Primero voy a compararlas para ver si los id estan bien.

In [298]:
resultados_comparacion = []


for index, row in df_2024.iterrows():
    id_actual = row['mal_id'] 
    nombre_actual = row['titles']  
    
    df_2019_row = df_2019[df_2019['animeID'] == id_actual]
    
    if not df_2019_row.empty:  
        nombre_2019 = df_2019_row[' name'].iloc[0]  
        
        
        if nombre_actual == nombre_2019:
            resultados_comparacion.append(True)
        else:
            resultados_comparacion.append(False)
    else:
        resultados_comparacion.append(False) 


df_2024['coincide_nombre'] = resultados_comparacion


print(df_2024[['mal_id', 'titles', 'coincide_nombre']].head())

   mal_id              titles  coincide_nombre
0    1550         Attack No.1             True
1    2406           Sazae-san             True
2    9895        The Bathroom             True
3    9163  Attack No.1 (1970)             True
4    7259          Kenju Giga             True


In [299]:
total_true = df_2024['coincide_nombre'].sum()
print(f'Número de coincidencias True: {total_true}')

Número de coincidencias True: 3920


##### Entonces podemos usar los dos DF ya que los id coinciden, hay muchos que no coinciden por que a lo mejor no salieron esos animes en 2019.

### Inspección y Limpieza 2024

In [300]:
df_2024.head()

,mal_id,titles,type,genre_1,genre_2,source,studios,episodes,rating,score,...,scored_by,rank,members,favorites,synopsis,status,airing,season,year,coincide_nombre
0,1550,Attack No.1,TV,Drama,Sports,Manga,Tokyo Movie Shinsha,104.0,PG-13 - Teens 13 or older,7.07,...,4117.0,4124.0,9979,72,Kozue is a middleschool girl and enthusiastic ...,Finished Airing,False,winter,1970.0,True
1,2406,Sazae-san,TV,Comedy,Slice of Life,4-koma manga,Eiken,NaN,G - All Ages,6.14,...,1869.0,9126.0,8403,38,The main character is a mother named Sazae-san...,Currently Airing,True,fall,1969.0,True
2,9895,The Bathroom,Movie,Avant Garde,Comedy,Original,Kuri Jikken Manga Koubou,1.0,R - 17+ (violence & profanity),4.22,...,1003.0,13223.0,1779,0,A surrealistic short from minimalist cartoonis...,NaN,NaN,NaN,NaN,True
3,9163,Attack No.1 (1970),Movie,Drama,Sports,Unknown,NaN,1.0,PG-13 - Teens 13 or older,6.39,...,444.0,7795.0,1410,3,"The first recap film of Attack No.1 , it cover...",NaN,NaN,NaN,NaN,True
4,7259,Kenju Giga,Movie,NaN,NaN,Original,NaN,1.0,PG-13 - Teens 13 or older,4.92,...,595.0,12800.0,1206,0,"A dog race is interrupted by a ringmaster, who...",Finished Airing,False,NaN,NaN,True


In [301]:
df_2024.columns
#La columnas ya estan limpias porque las limpié en el anterior para poder hacer bien la busqueda de información y el merge.

Index(['mal_id', 'titles', 'type', 'genre_1', 'genre_2', 'source', 'studios',
       'episodes', 'rating', 'score', 'audicence_rank', 'scored_by', 'rank',
       'members', 'favorites', 'synopsis', 'status', 'airing', 'season',
       'year', 'coincide_nombre'],
      dtype='object')

In [302]:
df_2024 = df_2024.drop(['members','favorites'],axis=1)

In [303]:
df_2024.shape

(5056, 19)

In [304]:
df_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5056 entries, 0 to 5055
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mal_id           5056 non-null   int64  
 1   titles           5056 non-null   object 
 2   type             5055 non-null   object 
 3   genre_1          4811 non-null   object 
 4   genre_2          3949 non-null   object 
 5   source           5056 non-null   object 
 6   studios          4544 non-null   object 
 7   episodes         4539 non-null   float64
 8   rating           5022 non-null   object 
 9   score            4817 non-null   float64
 10  audicence_rank   5056 non-null   int64  
 11  scored_by        4817 non-null   float64
 12  rank             4786 non-null   float64
 13  synopsis         5009 non-null   object 
 14  status           3121 non-null   object 
 15  airing           3121 non-null   object 
 16  season           1727 non-null   object 
 17  year          

In [305]:
df_2024.describe().T

,count,mean,std,min,25%,50%,75%,max
mal_id,5056.0,13436.020372,16425.207730,1.0,1427.50,4526.00,23347.50,58391.00
episodes,4539.0,18.509363,62.329765,1.0,1.00,12.00,23.00,1787.00
score,4817.0,7.029498,0.931790,2.0,6.46,7.11,7.67,9.38
audicence_rank,5056.0,5827.900514,5668.755969,1.0,1049.75,3843.00,9501.50,26508.00
scored_by,4817.0,122923.990450,274915.307253,105.0,1723.00,17975.00,118269.00,2795232.00
rank,4786.0,4862.653155,4240.655440,1.0,1320.00,3877.50,7454.00,20609.00
year,1727.0,1993.484655,11.931859,1969.0,1986.00,1996.00,2003.00,2011.00


In [306]:
df_2024.describe(include='object').T

,count,unique,top,freq
titles,5056,4516,Sazae-san,126
type,5055,8,TV,2861
genre_1,4811,21,Action,1814
genre_2,3949,20,Adventure,769
source,5056,17,Manga,2147
studios,4544,338,Toei Animation,487
rating,5022,6,PG-13 - Teens 13 or older,2444
synopsis,5009,4442,The main character is a mother named Sazae-san...,126
status,3121,2,Finished Airing,2616
airing,3121,2,False,2616


In [307]:
df_2024.dtypes

mal_id               int64
titles              object
type                object
genre_1             object
genre_2             object
source              object
studios             object
episodes           float64
rating              object
score              float64
audicence_rank       int64
scored_by          float64
rank               float64
synopsis            object
status              object
airing              object
season              object
year               float64
coincide_nombre       bool
dtype: object

In [308]:
df_2024.nunique()

mal_id             4516
titles             4516
type                  8
genre_1              21
genre_2              20
source               17
studios             338
episodes            160
rating                6
score               472
audicence_rank     4358
scored_by          3822
rank               3844
synopsis           4442
status                2
airing                2
season                4
year                 43
coincide_nombre       2
dtype: int64

In [309]:
#mal_id pone que hay 4516 unicos, significa que hay animes repetidos.
df_2024.duplicated().sum()

540

In [310]:
df_2024 = df_2024.drop_duplicates(keep='first')

In [311]:
df_2024['source'].unique()

array(['Manga', '4-koma manga', 'Original', 'Unknown', 'Novel',
       'Picture book', 'Music', 'Other', 'Book', 'Radio', 'Light novel',
       'Game', 'Mixed media', 'Visual novel', 'Card game', 'Web manga',
       'Web novel'], dtype=object)

In [312]:
sustitucion = {
    '4-koma manga':'Manga',
    'Web manga':'Manga',
    'Web novel':'Novel',
    }
df_2024['source'] = df_2024['source'].replace(sustitucion)
df_2024['source'].unique()

array(['Manga', 'Original', 'Unknown', 'Novel', 'Picture book', 'Music',
       'Other', 'Book', 'Radio', 'Light novel', 'Game', 'Mixed media',
       'Visual novel', 'Card game'], dtype=object)

In [313]:
#Es lo mismo que status
df_2024['airing'].unique()

array([False, True, nan], dtype=object)

In [314]:
df_2024 = df_2024.drop('airing',axis=1)

In [315]:
sustitucion1 = {
    'PG-13 - Teens 13 or older':'Teens or older',
    'G - All Ages':'All Ages',
    'R - 17+ (violence & profanity)':'17+',
    'PG - Children':'Children',
    'R+ - Mild Nudity':'Mild Nudity',
    'Rx - Hentai':'+18'
    }
df_2024['rating'] = df_2024['rating'].replace(sustitucion1)
df_2024['rating'].unique()

array(['Teens or older', 'All Ages', '17+', 'Children', 'Mild Nudity',
       nan, '+18'], dtype=object)

In [316]:
#Puede ser que sea por que son temporadas distintas de ciertos animes pero les dan la misma sinopsis, vamos a ver.
df_2024['synopsis'].duplicated().sum()

73

In [ ]:
duplicados = df_2024[df_2024['synopsis'].duplicated(keep=False)]
duplicados[['mal_id', 'titles', 'synopsis']].head(88)
#Vale es lo que pensaba, es por segundas temporadas o por falta de sinopsis solo ponen el autor...

In [317]:
df_2024.isna().sum()

mal_id                0
titles                0
type                  1
genre_1             245
genre_2            1057
source                0
studios             512
episodes             17
rating               34
score               239
audicence_rank        0
scored_by           239
rank                270
synopsis             47
status             1892
season             3286
year               3286
coincide_nombre       0
dtype: int64

In [318]:
#No me interesan las rows que no tienen score
df_2024 = df_2024.dropna(subset=['score'])

In [319]:
#Tampoco las que no tienen rank
df_2024 = df_2024.dropna(subset=['rank'])

In [320]:
#Me faltan muchos en año asi que voy a utilizar los años que vienen en el otro pos si puedo rellenar algunas, el resto iran como unknown.
df_2019['aired_year'] = df_2019[' aired'].astype(str).str.extract(r'(\d{4})')
df_2019['aired_year'].head()

0    1998
1    2001
2    1998
3    2002
4    2004
Name: aired_year, dtype: object

In [321]:
#Había que limpiarlos primero y aqui los meto.
id_to_year = df_2019.set_index('animeID')['aired_year']
df_2024['year'] = df_2024['year'].fillna(df_2024['mal_id'].map(id_to_year))

In [322]:
def extraer_estacion(aired_str):
    try:
        aired_dict = ast.literal_eval(aired_str)
        from_date = aired_dict.get('from')
        if from_date:
            mes = int(from_date[5:7])
            if mes in [12, 1, 2]:
                return 'winter'
            elif mes in [3, 4, 5]:
                return 'spring'
            elif mes in [6, 7, 8]:
                return 'summer'
            elif mes in [9, 10, 11]:
                return 'fall'
    except:
        return np.nan
    return np.nan


In [323]:
#Voy a hacer lo mismo con estas, pero necesito la funcion para colocar el mes en distintas estaciones.
df_2019['aired_season'] = df_2019[' aired'].apply(extraer_estacion)

In [324]:
id_to_season = df_2019.set_index('animeID')['aired_season'].to_dict()

In [325]:
df_2024['season'] = df_2024.apply(lambda row: id_to_season.get(row['mal_id']) if pd.isna(row['season']) else row['season'], axis=1)

In [326]:
#Quitamos ya la synopsis, solo la necesitaba para ver si gracias a ellas conseguia identificar nulos.
df_2024 = df_2024.drop('synopsis', axis=1)

In [327]:
#He solucionado los que podia solucionar con información del otro data frame, el resto unknown.
# Para columnas de texto.
for col in ['genre_1', 'genre_2', 'studios', 'rating', 'status', 'season', 'type']:
    df_2024[col] = df_2024[col].fillna('Unknown')

# Para columnas numéricas.
for col in ['episodes', 'year']:
    df_2024[col] = df_2024[col].fillna(-1)

In [328]:
df_2024.isna().sum()

mal_id             0
titles             0
type               0
genre_1            0
genre_2            0
source             0
studios            0
episodes           0
rating             0
score              0
audicence_rank     0
scored_by          0
rank               0
status             0
season             0
year               0
coincide_nombre    0
dtype: int64

In [329]:
df_2024.dtypes

mal_id               int64
titles              object
type                object
genre_1             object
genre_2             object
source              object
studios             object
episodes           float64
rating              object
score              float64
audicence_rank       int64
scored_by          float64
rank               float64
status              object
season              object
year                object
coincide_nombre       bool
dtype: object

In [330]:
#Despues de quitar los nulos ya puedo convertir los float a int sin problema.
cols_to_convert = ['episodes', 'audicence_rank', 'scored_by', 'rank']
df_2024[cols_to_convert] = df_2024[cols_to_convert].astype(int)

In [331]:
#Después de rellenarlo con los strings de el otro data set también puedo ya convertirlo a int sin problema.
df_2024['year'] = df_2024['year'].astype(int)

In [332]:
df_2024.dtypes

mal_id               int64
titles              object
type                object
genre_1             object
genre_2             object
source              object
studios             object
episodes             int32
rating              object
score              float64
audicence_rank       int32
scored_by            int32
rank                 int32
status              object
season              object
year                 int32
coincide_nombre       bool
dtype: object

In [333]:
#Limpito!!

### Inspección y Limpieza 2019

In [334]:
df_2019.columns

Index(['animeID', ' name', ' title_english', ' title_japanese',
       ' title_synonyms', ' type', ' source', ' producers', ' genre',
       ' studio', ' episodes', ' status', ' airing', ' aired', ' duration',
       ' rating', ' score', ' scored_by', ' rank', ' popularity', ' members',
       ' favorites', ' synopsis', ' background', ' premiered', ' broadcast',
       ' related', 'aired_year', 'aired_season'],
      dtype='object')

In [336]:
#Voy a empezar con las columnas ya que este dataframe solo lo necesito para compararlo con el otro, no necesito más informacion adicional.
df_2019.drop([' title_english', ' title_japanese',
       ' title_synonyms', ' type', ' source', ' producers', ' genre',
       ' studio', ' episodes', ' status', ' airing', ' aired', ' duration',
       ' members', ' favorites', ' synopsis', ' background', ' premiered', ' broadcast',
       ' related', 'aired_year', 'aired_season',' rating'], axis=1, inplace=True)

In [337]:
df_2019.head()

,animeID,name,score,scored_by,rank,popularity
0,1,Cowboy Bebop,8.81,405664.0,26.0,39.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.41,120243.0,164.0,449.0
2,6,Trigun,8.30,212537.0,255.0,146.0
3,7,Witch Hunter Robin,7.33,32837.0,2371.0,1171.0
4,8,Bouken Ou Beet,7.03,4894.0,3544.0,3704.0


In [338]:
df_2019.shape

(15278, 6)

In [339]:
df_2019.describe().T

,count,mean,std,min,25%,50%,75%,max
animeID,15278.0,19002.639416,13706.437012,1.0,5209.25,18308.00,33025.75,39199.0
score,14778.0,6.284508,1.073349,1.0,5.60,6.37,7.05,10.0
scored_by,15273.0,12107.014601,46929.422586,0.0,47.00,470.00,3857.00,1107955.0
rank,13669.0,6917.408004,4003.814986,0.0,3448.00,6932.00,10416.00,13838.0
popularity,15273.0,7741.991292,4482.323035,0.0,3855.00,7769.00,11627.00,15474.0


In [340]:
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15278 entries, 0 to 15277
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   animeID      15278 non-null  int64  
 1    name        15278 non-null  object 
 2    score       14778 non-null  float64
 3    scored_by   15273 non-null  float64
 4    rank        13669 non-null  float64
 5    popularity  15273 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 716.3+ KB


In [ ]:
#Voy a limpiar los nombres de las columnas.
df_2019.columns

Index(['animeID', ' name', ' score', ' scored_by', ' rank', ' popularity'], dtype='object')

In [343]:
df_2019.columns = df_2019.columns.str.strip() 
df_2019.rename(columns={
    'animeID': 'mal_id',
    'name': 'titles',
    'popularity': 'audience_rank'
}, inplace=True)
df_2019.columns

Index(['mal_id', 'titles', 'score', 'scored_by', 'rank', 'audience_rank'], dtype='object')

In [345]:
#Solo quiero las que tenga un mal_id que coincida.
df_2019 = df_2019[df_2019['mal_id'].isin(df_2024['mal_id'])]

In [346]:
df_2019.shape

(3569, 6)

In [354]:
df_2019.isna().sum()

mal_id           0
titles           0
score            0
scored_by        0
rank             0
audience_rank    0
dtype: int64

In [353]:
#No me interesan las rows que no tienen score ni rank
df_2019 = df_2019.dropna(subset=['score'])
df_2019 = df_2019.dropna(subset=['rank'])

In [ ]:
#Limpito!

### Insights